In [8]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib as plt
import requests
from urllib.parse import urlencode 
from datetime import timedelta
import json
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import calendar
import scipy.stats as st
import statsmodels.stats.api as sms
from tqdm import tqdm 

Подгружаем данные из Яндекс Диска. Для учебных целей в случае, если не доступен диск, материалы берутся из каталога с *.py-файлом.

In [9]:
try:
    groups_part_1 = get_df_yadisk('https://disk.yandex.ru/d/UhyYx41rTt3clQ', ';')
    groups_part_2 = get_df_yadisk('https://disk.yandex.ru/d/5Kxrz02m3IBUwQ', ',')
    active_user = get_df_yadisk('https://disk.yandex.ru/d/Tbs44Bm6H_FwFQ', ',')
    checks = get_df_yadisk('https://disk.yandex.ru/d/pH1q-VqcxXjsVA', ';')
except:
    groups_part_1 = pd.read_csv('2_groups.csv', sep=';')
    groups_part_2 =  pd.read_csv('2_group_add.csv', sep=',')
    active_user = pd.read_csv('2_active_studs.csv', sep=',')
    checks =  pd.read_csv('2_checks.csv', sep=';')

Предполагаем, что исследование мы проводим после того, как нам прислали файл **2_group_add.csv**. В связи с этим, первым делом мы объединяем **groups_part_1** и **groups_part_2**. Также переименовываем колонки, с целью дальнейшей простоты объединения файлов.

In [10]:
groups_part = pd.concat([groups_part_1, groups_part_2])
groups_part = groups_part.rename(columns={groups_part.columns[0]: "student_id", groups_part.columns[1]: "grp"})
groups_part.head()

,student_id,grp
0,1489,B
1,1627,A
2,1768,B
3,1783,B
4,1794,A


Добавляем в таблицы `с информацией о пользователях, которые зашли на платформу в дни проведения эксперимента`и `с информацией об оплатах пользователей в дни проведения эксперимента`, метки для дальнейшего использования их при объединении всех таблиц.

In [11]:
active_user['activ'] = 1
checks['check'] = 1

Объединяем таблицы с помощью **LEFT JOIN**, в связи с тем, что нам нужны только пользователи, которые входят группы А и B.

In [12]:
A_B_testing_full =  groups_part.merge(checks, how='left', on='student_id') \
            .merge(active_user, how='left', on='student_id')
A_B_testing_full.head()

,student_id,grp,rev,check,activ
0,1489,B,NaN,NaN,NaN
1,1627,A,990.0,1.0,1.0
2,1768,B,NaN,NaN,NaN
3,1783,B,NaN,NaN,NaN
4,1794,A,NaN,NaN,NaN


Заменяем значение **NaN** на **0**.

In [13]:
A_B_testing_full = A_B_testing_full.fillna(0)
A_B_testing_full

,student_id,grp,rev,check,activ
0,1489,B,0.0,0.0,0.0
1,1627,A,990.0,1.0,1.0
2,1768,B,0.0,0.0,0.0
3,1783,B,0.0,0.0,0.0
4,1794,A,0.0,0.0,0.0
...,...,...,...,...,...
74571,200247820,B,0.0,0.0,0.0
74572,201032527,B,0.0,0.0,0.0
74573,201067612,B,0.0,0.0,0.0
74574,201067653,B,0.0,0.0,0.0


Смотря на соотношение пользователей по группам, наблюдаем, что в `группе А` **~20%** всех участников теста, а соотвественно в `группе В` **~80%** участников.

In [14]:
A_B_testing_full.grp.value_counts(normalize=True)

B    0.80298
A    0.19702
Name: grp, dtype: float64

Проверяя данные на целостность,мы видим, что есть пользователи, которые купили услугу, но не делали никаких действий на сайте в дни проведения эксперимента. В связи с этим, можно предложить, что наша услуга подписочная, и у данных пользователей покупка совершилась автоматически.

In [15]:
A_B_testing_full[(A_B_testing_full.activ == 0) & (A_B_testing_full.check > 0)]

,student_id,grp,rev,check,activ
39,3185,B,690.0,1.0,0.0
121,25973,B,690.0,1.0,0.0
125,26280,B,690.0,1.0,0.0
223,100645,B,630.0,1.0,0.0
254,102534,B,580.0,1.0,0.0
...,...,...,...,...,...
72452,5291900,B,199.0,1.0,0.0
72753,5331842,B,290.0,1.0,0.0
73632,5486319,B,199.0,1.0,0.0
74049,5562806,B,398.0,1.0,0.0


В связи с тем, что мы оцениваем новую механику оплаты услуг, нам потребуются только пользователи, которые заходили на сайт и потенциально могли воспользоваться нашей услугой. Поэтому пользователи, которые не заходили на наш сайт, либо пользователи с подпиской в анализ `не входят`.

In [13]:
A_B_testing_activ = A_B_testing_full[A_B_testing_full['activ'] == 1]
A_B_testing_activ.grp.value_counts(normalize=True)

B    0.81561
A    0.18439
Name: grp, dtype: float64

Проверяем есть ли у нас пересечение пользователей в тестовой и контрольной группах.

In [14]:
t = A_B_testing_activ.groupby('student_id')['grp'].nunique().reset_index()
t[t['grp'] > 1]

,student_id,grp


Пересечений **НЕТ**.

За основу мы возьмём метрику `конверсии продаж`. Для оценки новой механики оплаты целесообразно взять именно эту метрику, так как нам важно, чтобы пользователь либо опробовал новую механику, либо совсем отказался от неё. 

In [15]:
temp = A_B_testing_activ.groupby('grp')[['check','rev']].agg(['count','mean'])
temp

check             rev           
    count      mean count       mean
grp                                 
A    1538  0.050715  1538  47.347204
B    6803  0.046156  6803  58.058798

In [13]:
temp.columns = ['_'.join(el) for el in temp.columns.to_flat_index()]
temp = temp.reset_index()
temp

,grp,check_count,check_mean,rev_count,rev_mean
0,A,1538,0.050715,1538,47.347204
1,B,6803,0.046156,6803,58.058798


Для проверки разницы между двумя пропорциями пользователей может использоваться двухпропорционный Z-тест.

Вводим 2 гипотезы.

Н0: среднее группы А = среднему группы В

Н1: средние не равны

In [14]:
sms.proportions_ztest([A_B_testing_activ[A_B_testing_activ.grp == 'A'].check.sum(),A_B_testing_activ[A_B_testing_activ.grp == 'B'].check.sum()],[A_B_testing_activ[A_B_testing_activ.grp == 'A'].check.count(),A_B_testing_activ[A_B_testing_activ.grp == 'B'].check.count()])

(0.7629884495263746, 0.445470284371589)

### **Вывод:** в связи с показателем P-Value = ~0.45, мы не можем отклонить нулевую гипотезу о равенстве средних между группами А и В, так как значения не являются статистически значимыми. В связи с этим, запускать новую механику на всех пользователей не имеет смысла.